In [286]:
import json
import pandas as pd
import spacy
from spacy.lang.en import English
import numpy as np
import json
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report
nlp = spacy.load('en_core_web_lg')
import re
from sklearn.metrics.pairwise import cosine_similarity
import argparse

In [230]:
data_file = 'yummly.json'

In [231]:
#Reads the input file (here the json file) and creates a dataframe for id, cuisine and ingredients
def get_data(data_file):
    id, cuisine, ingredients=[], [], [] #creating lists for dataframe
    with open(data_file, 'r') as infile:#opens the data file as infile
          text = infile.read() #reads the data file in 'text'
#    df = pd.read_json('yummly.json')
    data = json.loads(text) #loads the text file in 'data'
    for i in data:
        id.append(i['id'])
        cuisine.append(i['cuisine'])
        ingredients.append(' '.join(i['ingredients']))
    infile.close()
    dat = {'Id' : id, 'Cuisine' : cuisine, 'Ingredients' : ingredients} #Creates a dataframe 
    df = pd.DataFrame(dat)
#     df = pd.json_normalize(data,  meta=['id', 'cuisine', 'ingredients'])
# #     #df.head(3)

    return df

In [232]:
get_data(data_file)

,Id,Cuisine,Ingredients
0,10259,greek,romaine lettuce black olives grape tomatoes ga...
1,25693,southern_us,plain flour ground pepper salt tomatoes ground...
2,20130,filipino,eggs pepper salt mayonaise cooking oil green c...
3,22213,indian,water vegetable oil wheat salt
4,13162,indian,black pepper shallots cornflour cayenne pepper...
...,...,...,...
39769,29109,irish,light brown sugar granulated sugar butter warm...
39770,11462,italian,KRAFT Zesty Italian Dressing purple onion broc...
39771,2238,irish,eggs citrus fruit raisins sourdough starter fl...
39772,41882,chinese,boneless chicken skinless thigh minced garlic ...


In [275]:
def model(X_train, X_test, Y_train):
  
    vector= CountVectorizer()
    X_train_v= vector.fit_transform(X_train)
#     print(vector.get_feature_names_out())
#     X_train_v.toarray()
    X_test_v = vector.transform(X_test)
    clf = MultinomialNB()
    clf.fit(X_train_v, Y_train)
    
    return(vector, clf)
    
    
    

In [335]:
def predict(ingredients, vector, model):
    new_ing = []
    new_ing.append(' '.join(ingredients))
    ip = vector.transform(new_ing)
    
    pred = model.predict(ip)
    
    ipr = {'Id' : 'input', 'Cuisine' : pred[0], 'Ingredients' : new_ing[0]}
    print('Cuisine predicted based on the input ingredients: '+ pred[0])
    

    
    return(pred, ipr)

In [300]:
# def get_recipes(df, matrix, ingredients, pred):

#     doc_sim = cosine_similarity(matrix)
#     doc_sim_df = pd.DataFrame(doc_sim)
#     #print(doc_sim_df.head())
    
#     #Get index of 'User_input'
#     recipes_list = df['Id'].values
#     recipe_idx = np.where(recipes_list == 'User_input')[0][0]

#     #Get list of cosine similarities for user input/sort and store the cosine similarity score
#     recipe_similarities = doc_sim_df.iloc[recipe_idx].values
    
#     #Create / open file to write results to
#     f = open('docs/results.txt', 'w+')
#     f.write('\nInput Ingredients: ' + str(ingredients) + '\n')
#     f.write('Predicted Cuisine: ' + str(pred[0]) + '\n')
#     f.write('Closest 5 recipes: \n')

#     #Print top 5 recipes and cosine similarity distance
#     similar_recipe_idxs = np.argsort(-recipe_similarities)[1:6]
#     print('Closest 5 recipes: ')
#     i = 1
#     for idx in similar_recipe_idxs:
#         f.write(str(i) + '. Recipe Id: ' + str(recipes_list[idx]) + ' (' + str(doc_sim_df[idx][5000]) + ') \n')       
#         print(str(i) + '. Recipe Id: ' + str(recipes_list[idx]) + ' (' + str(doc_sim_df[idx][5000]) + ')')
#         i = i + 1
    
#     f.close()
    

In [428]:
# def similarity(df, doc_mat, ingredients, pred):
# #     vector1 = CountVectorizer()
# #     sparse_matrix = vector1.fit_transform(pred)
# #     doc_term_matrix = sparse_matrix.todense()
# #     df_similar = pd.DataFrame(doc_term_matrix, columns = vector1.get_feature_names_out())
# #     dj = pd.DataFrame(cosine_similarity(df_similar,dense_output = True))
    
#     similar = cosine_similarity(doc_mat)
#     similar_df = pd.DataFrame(similar)
    
#     recipies = df['Id'].values
#     recipe_ids = np.where(recipies == 'User_input')[0][0]
#     recipe_similarities = similar_df.iloc[recipe_ids].values
    
# #     res = open('results.txt', 'w+')
# #     res.write('\nInput Ingredients: ' + str(ingredients) + '\n')
# #     res.write('Predicted Cuisine: ' + str(pred[0]) + '\n')
# #     res.write('Closest recipes: \n')
#     print('\nInput Ingredients: ' + str(ingredients) + '\n')
#     print('Cuisine Predicted: ' + str(pred[0]) + '\n')

#     #Print top 5 recipes and cosine similarity distance
#     sim_recipe_ids = np.argsort(-recipe_similarities)[1:6]
#     print('5 Closest Recipies: ')
#     i = 1
#     for idx in sim_recipe_ids:
# #         res.write(str(i) + '. Recipe Id: ' + str(recipies[idx]) + ' (' + str(similar_df[idx][4000]) + ') \n')
# #         str(i) + '. Recipe Id: ' + str(recipies[idx]) + ' (' + str(similar_df[idx][4000]) + ') \n'
#         print(str(i) + '. Recipe Id: ' + str(recipies[idx]) + ' (' + str(similar_df[idx][4000]) + ')')
#         i = i + 1
# #     res.close()


In [429]:
def similarity(df, doc_mat, ingredients, pred):

    similar = cosine_similarity(doc_mat)
    similar_df = pd.DataFrame(similar)
    
    recipies = df['Id'].values
    recipe_ids = np.where(recipies == 'User_input')[0][0]
    recipe_similarities = similar_df.iloc[recipe_ids].values
    

    print('\nInput Ingredients: ' + str(ingredients) + '\n')
    print('Cuisine Predicted: ' + str(pred[0]) + '\n')

    #Printing 5 closest recipies and the cosine similarity distance
    sim_recipe_ids = np.argsort(-recipe_similarities)[1:6]
    print('5 Closest Recipies: ')
    i = 1
    for k in sim_recipe_ids:
        print(str(i) + '. Recipe Id: ' + str(recipies[k]) + ' (' + str(similar_df[k][4000]) + ')')
        i = i + 1

In [389]:
ingredients = ['taco', 'tortillas', 'guacamole', 'rice']

In [224]:
def main(ingredients):
    
          
    #Reads .json file and returns dataframe of id, cuisine, and ingredients
    get_data('yummly.json')

    #Split the data into 70-30 porportion for training and testing 
    X = df['Ingredients']
    Y = df['Cuisine']
    Z = df['Id']
    X_train, X_test, Y_train, Y_test, Z_train, Z_test = train_test_split(X, Y, Z,test_size = 0.3)     
    
    #Get vecorizer and model to predict cuisine
    vector, clf = model(X_train, X_test, Y_train)

    #Predict cuisine from user input
    pred, ipr = predict(ingredients, vector, clf)

    #Select closest N recipes - prints results to terminal and file
    data = {'Id' : X_train, 'Cuisine' : Y_train, 'Ingredients' : Z_train}
    tr_df = pd.DataFrame(data)
     
    #Add user input to train data
    tr_df = tr_df.append(input_row, ignore_index = True)
    tr_features = cv.fit_transform(tr_df['Ingredients'])
    get_recipes(tr_df, tr_features, ingredients, cuisine)


In [301]:
def test_predict():

    data_file = 'yummly.json'
    ingredients = ['eggs', 'soy', 'cheese', 'beef', 'flour','ham']

    #Parse .json file into df
    df = get_data(data_file)

    #Splitting df into 30% test and 70% train dataset 
    X = df['Ingredients']
    Y = df['Cuisine']
    Z = df['Id']
    X_train, X_test, Y_train, Y_test, Z_train, Z_test = train_test_split(X,Y,Z,test_size = 0.3)     

    #Get vectorizer and model
    vector, clf = model(X_train, X_test, Y_train)
    #vector = model(X_train, X_test, Y_train)
#     print(ingredients)  
    print()
    #Predict cuisine from user input
    pred, ipr = predict(ingredients, vector, clf)
    
    assert pred!=None

In [302]:
test_predict()


Cuisine predicted based on the input ingredients: southern_us


In [430]:
def test_similarity():

    data_file = 'yummly.json'
    ingredients = ['tomatoes', 'soy', 'cheese', 'beef', 'salt','ham']
    
    #Parse data file into dataframe
    df = get_data(data_file)

    #Applying test train split to df
    X = df['Ingredients']
    Y = df['Cuisine']
    Z = df['Id']
    X_train, X_test, Y_train, Y_test, Z_train, Z_test = train_test_split(X,Y,Z,test_size = 0.3)     

    #Get vectorizer and model
    vector, clf = model(X_train, X_test, Y_train)
#     print(ingredients)  
    print()
    pred, ipr = predict(ingredients, vector, clf)
    
    data = {'Id' : Z_train, 'Cuisine' : Y_train, 'Ingredients' : X_train}
    trn = pd.DataFrame(data)
    
    trn = trn.append(ipr, ignore_index = True)
    trn_vector = vector.fit_transform(tr_df['Ingredients'])
    
    similarity(trn, trn_vector, ingredients, pred)


In [431]:
test_similarity()


Cuisine predicted based on the input ingredients: italian


C:\Users\Kaustubh\AppData\Local\Temp\ipykernel_46768\1475077313.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tr_df = tr_df.append(ipr, ignore_index = True)



Input Ingredients: ['tomatoes', 'soy', 'cheese', 'beef', 'salt', 'ham']

Cuisine Predicted: italian

5 Closest Recipies: 
1. Recipe Id: 11231 (0.17025130615174977)
2. Recipe Id: 4896 (0.18677184190940715)
3. Recipe Id: 21651 (0.11785113019775795)
4. Recipe Id: 41346 (0.0468292905790847)
5. Recipe Id: 17172 (0.0936585811581694)
